In [1]:
n_cores = -1

In [2]:
import numpy as np
import pandas as pd

In [3]:
data = np.load('inga_out.npz')

In [4]:
mass = data['arr_0']
mass

array([ 552.293448,  338.046024,  724.221464, ...,  328.072907,
        262.19328 ,  636.278192])

In [5]:
name = data['arr_1']
name

array(['UNPD98266', 'UNPD207163', 'UNPD3499', ..., 'UNPD98267',
       'UNPD47332', 'UNPD101003'], dtype=object)

In [6]:
massabund = data['arr_2']
massabund

array([[  41.00329 ,    1.688456],
       [  43.01894 ,    2.135631],
       [  55.01894 ,    1.105409],
       ..., 
       [ 549.23414 ,   24.377134],
       [ 551.24979 ,   22.666363],
       [ 591.2447  ,    5.496205]])

In [7]:
blockind = data['arr_3']
blockind

array([     0,      0,      0, ..., 220988, 220988, 220988], dtype=uint32)

### Equal-width Binning

In [12]:
def equalWidth(name, massabund, blockind, threshold):
    """
    Using equal-width strategy to generate matrix.
    """
    values = massabund[:,0]
    start = values.min() // threshold * threshold
    end = values.max() // threshold * threshold + threshold
    valueRange = np.arange(start, end, threshold)
    
    features = generateMatrix(massabund, blockind, valueRange)
    np.save('equalWidth_'+str(threshold), features)
    
#     result = generateDataFrame(name, newfeatures)
#     result.to_csv('equalWidth_'+str(threshold)+'.csv', index=None)
    
    return features

In [13]:
def generateMatrix(massabund, blockind, valueRange):
    """
    Generating the corresponding matrix based on the valueRange.
    """
    result = np.zeros([blockind.max()+1, valueRange.size])
    
    previousRow = 0
    previousEnd = 0
    for i in range(blockind.size):
        row = blockind[i]
        v = massabund[i]
        m = v[0]
        a = v[1]
        if row == previousRow:
            start = previousEnd
        else:
            start = 0
        for col in range(start, valueRange.size):
            upperLim = valueRange[col] 
            if (m <= upperLim):
                result[row,col] = a
                previousRow = row
                previousEnd = col
                break
                
    goodCol = result.sum(axis=0) > 0
    newresult = result[:,goodCol]
    
    return newresult

In [14]:
def generateDataFrame(name, features):
    """
    Taking name and features to generate the corresponding dataframe.
    """
    mat = np.concatenate([name.reshape(-1,1), features], axis=1)
    column_name = ['name']
    for i in range(features.shape[1]):
        column_name.append('fraction'+str(i+1))
    df = pd.DataFrame(mat, columns=column_name)
    return df

In [ ]:
for n in [1, 0.5, 0.1, 0.05, 0.01]:
    equalWidth(name, massabund, blockind, n)

### Equal-frequency Binning

Not work very well for the distribution.